In [1]:
import pathpyG as pp
from torch_geometric.utils import cumsum, coalesce, degree, sort_edge_index
import torch

from scipy.sparse.csgraph import bellman_ford, dijkstra
import numpy as np

from collections import defaultdict


from tqdm import tqdm

In [2]:
t_sp = pp.TemporalGraph.from_csv('sociopatterns_highschool_2013.tedges').to_undirected()
print(t_sp)
print(torch.unique(t_sp.data.t).size(0))

Temporal Graph with 327 nodes, 11636 unique edges and 377016 events in [1385982080.0, 1386345600.0]

Graph attributes
	dst		<class 'torch.Tensor'> -> torch.Size([377016])
	t		<class 'torch.Tensor'> -> torch.Size([377016])
	src		<class 'torch.Tensor'> -> torch.Size([377016])

1157


/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'dst', 't', 'src'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [3]:
c = pp.algorithms.centrality.temporal_closeness_centrality(t_sp, delta=3600)
print(c)

100%|██████████| 1157/1157 [01:21<00:00, 14.25it/s]


Created temporal event DAG with 754359 nodes and 31906926 edges
{'454': 41671.338095238054, '640': 42220.10476190471, '1': 31384.485714285773, '939': 43385.1666666666, '185': 46485.271428571396, '258': 42662.533333333326, '55': 41131.62698412695, '170': 43822.54999999994, '9': 55537.204761904715, '453': 42306.650000000016, '45': 54390.77142857138, '14': 42915.57142857141, '190': 51695.83809523803, '400': 37502.9365079365, '637': 40505.24126984126, '255': 43516.342857142794, '275': 53779.13333333323, '176': 52638.13333333328, '533': 36436.70952380954, '116': 46335.46666666662, '151': 46517.87142857134, '866': 51019.77619047604, '280': 40957.63095238092, '484': 38709.39523809525, '243': 42567.83809523808, '687': 43355.67142857139, '54': 44738.06666666662, '364': 43477.92142857135, '374': 41239.258730158705, '295': 37942.38968253968, '441': 44693.04761904753, '101': 46261.46984126977, '425': 40808.355411255354, '47': 34439.18333333336, '241': 40734.0880952381, '179': 51910.84285714278, '2

In [5]:
tedges = [('a', 'b', 1), ('b', 'c', 5), ('c', 'd', 9), ('c', 'e', 9),
              ('c', 'f', 11), ('f', 'a', 13), ('a', 'g', 18), ('b', 'f', 21),
              ('a', 'g', 26), ('c', 'f', 27), ('h', 'f', 27), ('g', 'h', 28),
              ('a', 'c', 30), ('a', 'b', 31), ('c', 'h', 32), ('f', 'h', 33),
              ('b', 'i', 42), ('i', 'b', 42), ('c', 'i', 47), ('h', 'i', 50)]
t = pp.TemporalGraph.from_edge_list(tedges)
c = pp.algorithms.centrality.temporal_closeness_centrality(t, 5)
print(c)

100%|██████████| 17/17 [00:00<00:00, 3506.77it/s]

Created temporal event DAG with 49 nodes and 47 edges
{'a': 12.0, 'b': 16.0, 'c': 16.0, 'd': 14.666666666666666, 'e': 14.666666666666666, 'f': 24.0, 'g': 14.666666666666666, 'h': 28.0, 'i': 24.0}


In [2]:
t = pp.TemporalGraph.from_edge_list([(0,1,0), (0,2,0), (1,2,1), (1,3,1), (3,4,2), (1,4,3)])
print(t)

Temporal Graph with 5 nodes, 6 unique edges and 6 events in [0.0, 3.0]

Graph attributes
	src		<class 'torch.Tensor'> -> torch.Size([6])
	dst		<class 'torch.Tensor'> -> torch.Size([6])
	t		<class 'torch.Tensor'> -> torch.Size([6])



/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'src', 'dst', 't'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [5]:
c = pp.algorithms.centrality.temporal_closeness_centrality(t, delta=1)
print(c)

100%|██████████| 4/4 [00:00<00:00, 262.99it/s]

Created temporal event DAG with 17 nodes and 15 edges
{0.0: 0.0, 1.0: 4.0, 2.0: 8.0, 3.0: 6.0, 4.0: 9.333333333333332}


In [3]:
# old code with explosive memory usage due to computation of all second-order edges irrespective of time stamps
def lift_order_not_efficient(g: pp.TemporalGraph, delta=1):
    # first-order edge index
    edge_index, timestamps = sort_edge_index(g.data.edge_index, g.data.t)
    node_sequence = torch.arange(g.data.num_nodes, device=edge_index.device).unsqueeze(1)
    print(edge_index)
    # second-order edge index with time-respective filtering
    null_model_edge_index = pp.MultiOrderModel.lift_order_edge_index(edge_index, num_nodes=node_sequence.size(0))    
    # Update node sequences
    node_sequence = torch.cat([node_sequence[edge_index[0]], node_sequence[edge_index[1]][:, -1:]], dim=1)
    # Remove non-time-respecting higher-order edges
    time_diff = timestamps[null_model_edge_index[1]] - timestamps[null_model_edge_index[0]]
    non_negative_mask = time_diff > 0
    delta_mask = time_diff <= delta
    time_respecting_mask = non_negative_mask & delta_mask
    edge_index = null_model_edge_index[:, time_respecting_mask]
    return edge_index

In [4]:
# new memory-efficient code
def lift_order_efficient(g: pp.TemporalGraph, delta: int = 1):

    # first-order edge index
    edge_index, timestamps = g.data.edge_index, g.data.t
    # print(edge_index)

    indices = torch.arange(0, edge_index.size(1), device=g.data.edge_index.device)

    unique_t = torch.unique(timestamps, sorted=True)
    second_order = []

    # lift order: find possible continuations for edges in each time stamp
    for i in tqdm(range(unique_t.size(0))):
        t = unique_t[i]
        #print('timestamp index ', i)
        #print('timestamp ', t)
        
        # find indices of all source edges that occur at unique timestamp t
        src_time_mask = (timestamps == t)
        src_edges = edge_index[:,src_time_mask]
        src_edge_idx = indices[src_time_mask]
        #print(src_edges)
        #print(src_edge_idx)

        # find indices of all edges that can possibly continue edges occurring at time t for the given delta
        dst_time_mask = (timestamps > t) & (timestamps <= t+delta)
        dst_edges = edge_index[:,dst_time_mask]        
        dst_edge_idx = indices[dst_time_mask]
        #print(dst_edges)
        #print(dst_edge_idx)

        if dst_edge_idx.size(0)>0 and src_edge_idx.size(0)>0:

            # compute second-order edges between src and dst idx for all edges where dst in src_edges matches src in dst_edges        
            x = torch.cartesian_prod(src_edge_idx, dst_edge_idx).t()
            src_edges = torch.index_select(edge_index, dim=1, index=x[0])
            dst_edges = torch.index_select(edge_index, dim=1, index=x[1])
            #print(src_edges)
            #print(dst_edges)
            ho_edge_index = x[:,torch.where(src_edges[1,:] == dst_edges[0,:])[0]]
            second_order.append(ho_edge_index)
            #print(ho_edge_index) 
            
            # #print('dst', dst)
            # src_mask = (edge_index[:,mask][0]==dst)
            # ctd = edge_index[:,mask][:,src_mask]
            # #print('continuations', ctd)
            # ctd_indices = torch.where(edge_index[:,mask][0]==dst)[0]        
            # #print('ctd indx', ctd_indices)
            # count += ctd_indices.size(0)
    ho_index = torch.cat(second_order, dim=1)    
    return ho_index

In [14]:
def fo_nodes(ho_edge, g):
    src_edge = ho_edge[0]
    dst_edge = ho_edge[1]
    return g.data.edge_index[:,src_edge][0], g.data.edge_index[:,dst_edge][0], g.data.edge_index[:,dst_edge][1]


def temporal_shortest_paths_all(g: pp.TemporalGraph, delta: int):
    # generate temporal event DAG
    edge_index = lift_order_efficient(g, delta)

    # Add indices of first-order nodes as src and dst of paths in TEG
    src_edges_src = g.data.edge_index[0,:] + g.data.edge_index.size(1)
    src_edges_dst = torch.arange(0, g.data.edge_index.size(1))    
    dst_edges_src = torch.arange(0, g.data.edge_index.size(1))
    dst_edges_dst = g.data.edge_index[1,:] + 2*g.data.edge_index.size(1)

    src_edges = torch.stack([src_edges_src, src_edges_dst])
    dst_edges = torch.stack([dst_edges_src, dst_edges_dst])
    edge_index = torch.cat([edge_index, src_edges, dst_edges], dim=1)

    event_graph = pp.Graph.from_edge_index(edge_index)
    
    # initialize distance matrix 
    dist = torch.full((g.N, event_graph.N), float("inf"), device=g.data.edge_index.device)

    # predecessor lists
    pred = defaultdict(lambda: defaultdict(list))

    # Fastest known single source SP in DAG (Cormen, Leiserson): single scan of edges in DAG
    # trick: index of second-order nodes = topological sorting of event DAG assuming that edges are given in chronological order    
    # scan second-order nodes in topological order and relax distances between first-order nodes

    # TODO: correct algorithm
    for src in tqdm(g.nodes):
        dist[g.mapping.to_idx(src), g.mapping.to_idx(src) + g.data.edge_index.size(1)] = 0
        for v in event_graph.nodes:
            for w in event_graph.successors(v):
                dist[g.mapping.to_idx(src), w] = min(dist[g.mapping.to_idx(src), w], dist[g.mapping.to_idx(src), v]+1)
    
    dist_fo = dist[:,2*g.M:] - 1
    dist_fo.fill_diagonal_(0)
    return dist_fo, pred


def temporal_shortest_paths(g: pp.TemporalGraph, delta: int):
    # generate temporal event DAG
    edge_index = lift_order_efficient(g, delta)
    

    # Tadd indices of first-order nodes as src and dst of paths in TEG
    src_edges_src = g.data.edge_index[0,:] + g.data.edge_index.size(1)
    src_edges_dst = torch.arange(0, g.data.edge_index.size(1))    
    dst_edges_src = torch.arange(0, g.data.edge_index.size(1))
    dst_edges_dst = g.data.edge_index[1,:] + 2*g.data.edge_index.size(1)

    src_edges = torch.stack([src_edges_src, src_edges_dst])
    dst_edges = torch.stack([dst_edges_src, dst_edges_dst])
    edge_index = torch.cat([edge_index, src_edges, dst_edges], dim=1)

    event_graph = pp.Graph.from_edge_index(edge_index)
          
    m = event_graph.get_sparse_adj_matrix()
    dist, pred = dijkstra(m, directed=True, indices = np.arange(g.M, g.M+g.N),  return_predecessors=True, unweighted=True)
    dist_fo = dist[:,2*g.M:] - 1
    np.fill_diagonal(dist_fo, 0)
    pred_fo = pred[:,2*g.M:]
    return dist_fo, pred_fo


    
def temporal_closeness_centrality(g: pp.TemporalGraph, delta: int) -> dict:

    centralities = dict()
    dist, _ = temporal_shortest_paths(g, delta)
    for x in g.nodes:
        centralities[x] = sum((g.N - 1) / dist[np.arange(g.N)!=x, g.mapping.to_idx(x)])

    return centralities

In [15]:
temporal_shortest_paths_all(t, delta=1)

100%|██████████| 4/4 [00:00<00:00, 3057.63it/s]
5it [00:00, 1392.35it/s]


(tensor([[0., inf, inf, inf, inf],
         [inf, 0., inf, inf, inf],
         [inf, inf, 0., inf, inf],
         [inf, inf, inf, 0., inf],
         [inf, inf, inf, inf, 0.]]),
 defaultdict(<function __main__.temporal_shortest_paths_all.<locals>.<lambda>()>,
             {}))

In [ ]:
idx[:,1]

In [ ]:
edge_index = lift_order_efficient(t)
print(edge_index)

In [ ]:
print(t.data.edge_index)
dist, pred = temporal_shortest_paths(t, delta=1)

print(dist)
print(pred)

In [ ]:
dist[:,4]

In [ ]:
t.mapping.node_ids

In [ ]:
print(temporal_closeness_centrality(t, delta=1))
print(t.N)

In [ ]:
temporal_shortest_paths(t_sp, delta=3600)

In [ ]:
edge_index[0,:]

In [ ]:
t.data.edge_index[:,edge_index[0,:]][0]

In [ ]:
t.data.edge_index[:,edge_index[1,:]][1]

In [ ]:
#print(t.data.edge_index)
print(t_sp)
g = temporal_shortest_paths(t_sp, delta=300)

In [ ]:
indeg = degree(g.data.edge_index[1])
roots = torch.where(indeg==0)[0]
print(roots)

In [ ]:
def traverse(g, path):
    if g.get_successors(path[-1]).size(0) == 0:
        pass
    else:
        for w in g.successors(path[-1]):
            traverse(g, path + (w,))

In [ ]:
i = 0
for x in roots:
    print(x)
    traverse(g, (x,))

In [ ]:
ho_index = lift_order_not_efficient(t, delta=1)
print(ho_index)

In [ ]:
ho_index = lift_order_efficient(t, delta=1)
print(ho_index)

In [ ]:
print(t.data.edge_index)

In [ ]:
node_sequence = torch.arange(t.data.num_nodes, device=t.data.edge_index.device).unsqueeze(1)
print(node_sequence)
node_sequence = torch.cat([node_sequence[t.data.edge_index[0]], node_sequence[t.data.edge_index[1]][:, -1:]], dim=1)
print(node_sequence)

In [ ]:
lift_order_not_efficient(t_sp, delta=300)

In [ ]:
lift_order_efficient(t_sp, delta=300)

In [ ]:
lift_order_not_efficient(t_sp, delta=300)

In [ ]:
x = torch.cartesian_prod(torch.tensor([0,1]), torch.tensor([1,3])).t()
# edge 0 = 0->1
# edge 1 = 1->2
# edge 2 = 0->1

# combination 0,1:     0->1, 1->2
# combination 0,2:     0->1, 0->1
print(x)

In [ ]:
src_edges = torch.index_select(t.data.edge_index, dim=1, index=x[0])
print(src_edges)

In [ ]:
dst_edges = torch.index_select(t.data.edge_index, dim=1, index=x[1])
print(dst_edges)

In [ ]:
 #select all indices where 
torch.where(src_edges[1,:] == dst_edges[0,:])[0]
x[:,torch.where(src_edges[1,:] == dst_edges[0,:])[0]]